# start

In [1]:
prfx='0312_1'

In [2]:
import numpy as np
from collections import Counter, defaultdict
import torch
from transformers import *
from tqdm import tqdm
import gc
import pickle

import pandas as pd
pd.set_option('display.max_columns', 500)
import dask.dataframe as dd

import datetime
def dtnow(): return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')


HOME='/tmp/working/data/git/recsys20'
p_in=f'{HOME}/input'

# pretrained_weights='bert-base-multilingual-cased'
# bertmodel = BertModel.from_pretrained(pretrained_weights)
# tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=False)

# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/trn.tsv
#  148,075,238 data/trn.tsv
# (rcss20) ➜  recsys20 git:(master) ✗ wc -l input/val.tsv
#  15,127,684 data/val.tsv

# !head -500000 {p_in}/training.tsv > {p_in}/trn5e5.tsv
# !head -50000  {p_in}/val.tsv      > {p_in}/val5e4.tsv

In [3]:
cols=[
'text_tokens',
'hashtags',
'tweet_id',
'present_media',
'present_links',
'present_domains',
'tweet_type',
'language',
'timestamp',

'engaged_user_id',
'engaged_follower_count',
'engaged_following_count',
'engaged_is_verified',
'engaged_account_creation_time',

'engaging_user_id',
'engaging_follower_count',
'engaging_following_count',
'engaging_is_verified',
'engaging_account_creation_time',

'engagee_follows_engager',
'reply_engagement_timestamp',
'retweet_engagement_timestamp',
'retweet_with_comment_engagement_timestamp',
'like_engagement_timestamp',
]

col2i={v:k for k,v in enumerate(cols)}

cols_val=cols[:-4]
cols_time=[
'timestamp',
'engaged_account_creation_time',
'engaging_account_creation_time',
'reply_engagement_timestamp',
'retweet_engagement_timestamp',
'retweet_with_comment_engagement_timestamp',
'like_engagement_timestamp',
]

In [4]:
def readcols(cols2use):
    return pd.read_csv(f'{p_in}/trn.tsv',sep='\x01',encoding='utf8',header=None,
                usecols=[col2i[col] for col in cols2use],names=cols2use)

def readchnks(cols2use=cols, chunksize=1e6):
    return pd.read_csv(f'{p_in}/trn.tsv',sep='\x01',header=None,encoding='utf8',
                       usecols=[col2i[col] for col in cols2use],names=cols2use,
                       chunksize=chunksize)

# make maps

## val

In [4]:
%%time
dfval=pd.read_csv(f'{p_in}/val.tsv',sep='\x01',encoding='utf8',header=None,
                  usecols=range(1,len(cols_val)),names=cols_val[1:],)

CPU times: user 1min 11s, sys: 7.62 s, total: 1min 19s
Wall time: 1min 56s


In [5]:
f'len(dfval): {len(dfval):,}'

'len(dfval): 15,127,684'

In [6]:
unq_twts_val = dfval.tweet_id.unique()
unq_enged_usrs_val = dfval.engaged_user_id.unique()
unq_engng_usrs_val = dfval.engaging_user_id.unique()

nunq_twts_val = len(unq_twts_val)
nunq_enged_usrs_val = len(unq_enged_usrs_val)
nunq_engng_usrs_val = len(unq_engng_usrs_val)

print(f'nunq_twts_val: {nunq_twts_val:,}' )
print(f'nunq_enged_usrs_val: {nunq_enged_usrs_val:,}')
print(f'nunq_engng_usrs_val: {nunq_engng_usrs_val:,}')

nunq_twts_val: 10,799,267
nunq_enged_usrs_val: 4,281,756
nunq_engng_usrs_val: 7,662,966


In [7]:
i2twttyp=list(dfval.tweet_type.value_counts().index.values)
twttyp2i={v:k for k,v in enumerate(i2twttyp)}

In [8]:
dfval.language.value_counts()

D3164C7FBCF2565DDF915B1B3AEFB1DC    6477021
22C448FF81263D4BAF2A176145EE9EAD    2368013
06D61DCBBE938971E1EA0C38BD9B5446    1313750
ECED8A16BE2A5E8871FD55F4842F16B1    1087316
B9175601E87101A984A50F8A62A1C374     772950
                                     ...   
4690215948DBF6872B8ED1C2BC87B17E         81
D18801336202297E6484F634CAC6592E         58
2E18F6F53E3CF073911AF0A93BBE5373          4
B2235C8B73239FDC5780DD132419833A          3
515E873C86EE1577E75FA2387B7FA59E          1
Name: language, Length: 66, dtype: int64

In [9]:
i2lang=list(dfval.language.value_counts().index.values)
lang2i={v:k for k,v in enumerate(i2lang)}

In [11]:
hshtg2cnt=Counter(j for i in dfval.hashtags.dropna().apply(lambda x: x.split('\t')) for j in i)
i2hshtg=[o[0] for o in hshtg2cnt.most_common()]
hshtg2i={v:k for k,v in enumerate(i2hshtg)}

In [12]:
link2cnt=Counter(j for i in dfval.present_links.dropna().apply(lambda x: x.split('\t')) for j in i)
i2link=[o[0] for o in link2cnt.most_common()]
link2i={v:k for k,v in enumerate(i2link)}

In [13]:
link2cnt.most_common(10)

[('E522B6C629B01DA6B29DE65BF5DFA55B', 3597),
 ('979B4F7F5C3B6AC61479D7E0070A2D17', 3136),
 ('140BCA0056E50C79778369C95D6DDC63', 1483),
 ('6F4FC1683D0D6FEC76DDBE6C699BCF10', 1222),
 ('52B4B186BC13C290B696BD5D304DA89A', 999),
 ('4A939FCEAF42B180561F4B7D8ACE2406', 911),
 ('180511B4A939ADAD39FDB6604C1A9AC5', 899),
 ('FF3D932C8BDCBC65A6D0A834418B03FB', 897),
 ('EF4BB86374435F3327FA6DFD08350868', 846),
 ('608B880AE03A9C931AEF7F855D0118B4', 829)]

In [14]:
domn2cnt=Counter(j for i in dfval.present_domains.dropna().apply(lambda x: x.split('\t')) for j in i)
i2domn=[o[0] for o in domn2cnt.most_common()]
domn2i={v:k for k,v in enumerate(i2domn)}

In [15]:
domn2cnt.most_common(10)

[('3896E26D12C903F0A00B6B1BE9A9BEA3', 201179),
 ('E91CDEC8DC7ABF30592FA024616FF970', 129797),
 ('FECA6F2E8244F2294BD2CE957C0602A9', 96457),
 ('FCA72262B99BB78F010B25C882A0C93C', 55659),
 ('A32007FFB62B2DFC4E449BD37F985C53', 32502),
 ('9E34EBC679CA0BC677D198BEE3E37113', 31259),
 ('A898ECD422D0575A276A0D74CD01C867', 28438),
 ('BBA1D3359A23689C6783652EA73B12BA', 23125),
 ('D0120C5A771484D1765BBA60461666B9', 17456),
 ('979ECC05068307E0490EA1B9B9B44411', 17080)]

### save results

In [16]:
pickle.dump(i2twttyp,open(f'{p_in}/i2twttyp_{prfx}.p','wb'))
pickle.dump(i2lang,open(f'{p_in}/i2lang_{prfx}.p','wb'))
pickle.dump(i2hshtg,open(f'{p_in}/i2hshtg_{prfx}.p','wb'))
pickle.dump(i2link,open(f'{p_in}/i2link_{prfx}.p','wb'))
pickle.dump(i2domn,open(f'{p_in}/i2domn_{prfx}.p','wb'))

## trn

In [91]:
%%time
df_tweet_id=readcols(['tweet_id'])
# Wall time: 13min 13s
i2twt=df_tweet_id.tweet_id.unique()
# twt2i={v:k for k,v in enumerate(i2twt)}
del df_tweet_id; gc.collect()
pickle.dump(i2twt,open(f'{p_in}/i2twt_{prfx}.p','wb'))

CPU times: user 6min 30s, sys: 31.4 s, total: 7min 1s
Wall time: 13min 13s


In [5]:
%%time
df_user_id=readcols(['engaged_user_id','engaging_user_id'])
# Wall time: 14min 42s
i2usr=list(set(df_user_id.engaged_user_id)|set(df_user_id.engaging_user_id))
usr2i={v:k for k,v in enumerate(i2usr)}
# del df_user_id; gc.collect()
pickle.dump(i2usr,open(f'{p_in}/i2usr_{prfx}.p','wb'))

CPU times: user 7min 31s, sys: 35.5 s, total: 8min 7s
Wall time: 14min 42s


# prep data

## load maps

In [5]:
%%time
i2twttyp=pickle.load(open(f'{p_in}/i2twttyp_{prfx}.p','rb'))
i2lang=pickle.load(open(f'{p_in}/i2lang_{prfx}.p','rb'))
i2hshtg=pickle.load(open(f'{p_in}/i2hshtg_{prfx}.p','rb'))
i2link=pickle.load(open(f'{p_in}/i2link_{prfx}.p','rb'))
i2domn=pickle.load(open(f'{p_in}/i2domn_{prfx}.p','rb'))
i2twt=pickle.load(open(f'{p_in}/i2twt_{prfx}.p','rb'))
i2usr=pickle.load(open(f'{p_in}/i2usr_{prfx}.p','rb'))

In [9]:
def make_x2i(i2x): return {v:k for k,v in enumerate(i2x)}

In [10]:
%%time
twttyp2i=make_x2i(i2twttyp)
lang2i=make_x2i(i2lang)
hshtg2i=make_x2i(i2hshtg)
link2i=make_x2i(i2link)
domn2i=make_x2i(i2domn)
twt2i=make_x2i(i2twt)
usr2i=make_x2i(i2usr)

## map val data

In [7]:
%%time
dfval=pd.read_csv(f'{p_in}/val.tsv',sep='\x01',encoding='utf8',header=None,
                  usecols=range(1,len(cols_val)),names=cols_val[1:],)

CPU times: user 1min 14s, sys: 7.04 s, total: 1min 21s
Wall time: 1min 20s


In [12]:
dfval.head(5).engaged_user_id.map(usr2i)

0     7438379
1    26178136
2    17290440
3    29752193
4      936864
Name: engaged_user_id, dtype: int64

In [15]:
%%time
enged_usrs_val = dfval.engaged_user_id.map(usr2i)

CPU times: user 1min 25s, sys: 4.78 s, total: 1min 30s
Wall time: 1min 29s


In [16]:
%%time
engng_usrs_val = dfval.engaging_user_id.map(usr2i)

CPU times: user 1min 28s, sys: 4.74 s, total: 1min 33s
Wall time: 1min 33s


In [17]:
len(enged_usrs_val),len(engng_usrs_val)

(15127684, 15127684)

In [ ]:
set_enged_usrs_val = set(enged_usrs_val)
set_engng_usrs_val = set(engng_usrs_val)

In [ ]:
len(set_enged_usrs_val),len(set_engng_usrs_val)

In [ ]:
1